In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    access_token = userdata.get('DEFORMER_TOKEN')
    !pip install git+https://$access_token@github.com/ay94/deformer-extractor.git@error-handling

# Experiment Outputs for testing


In [ ]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler
base_folder = colab.init('My Drive')

2024-08-03 23:40:44 - INFO - PyTorch version 1.13.1 available.
2024-08-03 23:40:46 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/scripts/My Drive
2024-08-03 23:40:46 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [ ]:
import torch
def load_states():
    last_hidden_states = torch.load(config_path / 'last_hidden_states.pt')
    return last_hidden_states

def convert_data_to_list(data):
	converted_data = dict()
	for k, v in data.items():
		if not isinstance(v, torch.Tensor):
			converted_data[k] = v
		converted_data[k] = v.tolist()
	return converted_data


def convert_data_to_tensor(data):
    converted_data = dict()
    for k, v in data.items():
        # This line checks if the key should be excluded from conversion
        if k not in ['input_ids', 'attention_mask', 'last_hidden_states', 'losses', 'logits', 'labels']:
            converted_data[k] = v
        else:
            # This line converts the data to a tensor if the key is not in the exclusion list
            converted_data[k] = torch.tensor(v, dtype=torch.long)
    return converted_data



def save_model_outputs(config_path, data):
    filename = config_path / 'model_outputs.pth'
    torch.save(data, filename)

def load_model_outputs(config_path):
    filename = config_path / 'model_outputs.pth'
    loaded_data = torch.load(filename)
    return loaded_data


In [ ]:
config_path = base_folder / 'Final Year Experiments/Class Imbalance/1_fineTuning'
fh = FileHandler(config_path)

model_name='arabertv02'
data_name='ANERCorp_CamelLab'
training_outputs = fh.load_pickle(
            f"evalOutputs/{model_name}_{data_name}_regular_outputs.pkl"
        )

In [ ]:
NERData = dict()
NERData['splits'] = {'train': [{'id': sen[0], 'words': sen[1], 'tags': sen[2]}for sen in training_outputs.data['train']],
                     'test': [{'id': sen[0], 'words': sen[1], 'tags': sen[2]}for sen in training_outputs.data['test']]}
NERData['labels'] = training_outputs.data['labels']
NERData['labels_map'] = training_outputs.data['labels_map']
NERData['inv_labels'] = training_outputs.data['inv_labels']

# Annotations

In [ ]:
from experiment_utils.configurations import  TokenizationConfig, ClusteringConfig
from experiment_utils.tokenization import TokenizationWorkflowManager
corpora_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
corpora_fh = FileHandler(corpora_path)
corpora  = corpora_fh.load_json('corpora.json')


split = 'test'
config_path = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
config_fh = FileHandler(config_path)
config = config_fh.load_yaml('config.yaml')
tokenization_config = TokenizationConfig.from_dict(config.get('tokenization'))

clustering_config = ClusteringConfig.from_dict(config.get('clustering'))


In [ ]:
# NERData = corpora['ANERCorp_CamelLab']
tokenization_outputs = TokenizationWorkflowManager(NERData, tokenization_config)

2024-08-03 23:40:46 - INFO - Loading Tokenizer aubmindlab/bert-base-arabertv02
2024-08-03 23:40:47 - INFO - Loading Preprocessor aubmindlab/bert-base-arabertv02
2024-08-03 23:40:47 - INFO - Processing train split


  0%|          | 0/3402 [00:00<?, ?it/s]

2024-08-03 23:40:59 - INFO - Extracting train subwords


  0%|          | 0/3402 [00:00<?, ?it/s]

2024-08-03 23:41:00 - INFO - Processing test split


  0%|          | 0/961 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
analysis_df = pd.read_json(
	config_path / 'test_analysis_df.json',
 	lines=True
)
old_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_analysis_df.jsonl.gz',
    lines=True
    )



In [ ]:
old_df.columns

Index(['global_id', 'token_id', 'word_id', 'sen_id', 'token_ids', 'label_ids',
       'first_tokens_freq', 'first_tokens_consistency',
       'first_tokens_inconsistency', 'words', 'wordpieces', 'tokens',
       'first_tokens', 'truth', 'pred', 'agreement', 'losses', 'x', 'y',
       'tokenization_rate', 'token_entropy', 'word_entropy', 'tr_entity',
       'pr_entity', 'error_type', 'prediction_entropy', 'confidences',
       'variability', 'O', 'B-PERS', 'I-PERS', 'B-ORG', 'I-ORG', 'B-LOC',
       'I-LOC', 'B-MISC', 'I-MISC', '3_clusters', '4_clusters', '9_clusters'],
      dtype='object')

In [ ]:
subword_index = tokenization_outputs.train_subwords


In [ ]:
model_outputs = load_model_outputs(config_path)

In [ ]:
import numpy as np
import pandas as pd
from experiment_utils.analysis import  DataExtractor, UtilityFunctions, ClusterAnalysis




In [ ]:
flat_data = DataExtractor(tokenization_outputs.test, model_outputs).extract_features()


In [ ]:
cluster_analysis = ClusterAnalysis(flat_data, analysis_df, clustering_config)

In [ ]:
outputs = cluster_analysis.calculate()

2024-08-03 23:41:10 - INFO - Calculating Silhouette Score
2024-08-03 23:41:33 - INFO - Processing K=3
2024-08-03 23:41:37 - INFO - Processing K=4
2024-08-03 23:41:41 - INFO - Processing K=9


In [ ]:
tr_im = TrainingImpact(
            NERData['splits']['test'], tokenization_outputs,  'aubmindlab/bert-base-arabertv02', model.bert
        )

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tr_im.compute_attention_similarities(10)

Computing attention similarities:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
tr_im.compare_weights()

In [ ]:
analysis_df['global_ids']= UtilityFunctions.global_ids_from_df(analysis_df)

test = cluster_analysis.df.merge(analysis_df, on='global_ids', how='inner')

In [ ]:
analysis_df.head(10)

,labels,losses,token_ids,words,tokens,word_pieces,core_tokens,true_labels,pred_labels,sentence_ids,token_positions,token_selector_id,agreements,x,y
0,-100,0.000000,2,[CLS],[CLS],[CLS],[CLS],[CLS],[CLS],0,0,[CLS]@#0@#0,True,8.268111,-5.414730
1,5,0.305856,40508,الصالحية,الصالحية,[الصالحية],الصالحية,B-LOC,B-LOC,0,1,الصالحية@#1@#0,True,11.812889,20.155428
2,5,0.021982,32479,المفرق,المفرق,[المفرق],المفرق,B-LOC,B-LOC,0,2,المفرق@#2@#0,True,11.837891,20.212543
3,0,0.001037,19,-,-,[-],-,O,O,0,3,-@#3@#0,True,12.842813,1.836821
4,1,0.022747,33014,غيث,غيث,[غيث],غيث,B-PERS,B-PERS,0,4,غيث@#4@#0,True,3.389490,20.064877
5,2,0.013149,34675,الطراونة,الطر,"[الطر, ##او, ##نة]",الطر,I-PERS,I-PERS,0,5,الطر@#5@#0,True,0.894050,18.283405
6,-100,0.000000,43752,الطراونة,##او,"[الطر, ##او, ##نة]",IGNORED,IGNORED,IGNORED,0,6,IGNORED@#6@#0,True,1.451841,17.957733
7,-100,0.000000,466,الطراونة,##نة,"[الطر, ##او, ##نة]",IGNORED,IGNORED,IGNORED,0,7,IGNORED@#7@#0,True,2.013506,17.776596
8,0,0.000534,19,-,-,[-],-,O,O,0,8,-@#8@#0,True,12.798790,1.832167
9,0,0.000040,2279,أمر,أمر,[أمر],أمر,O,O,0,9,أمر@#9@#0,True,10.175797,6.479431


In [ ]:
ent = Entity(training_outputs.test_metrics.seq_output)

In [ ]:
training_outputs.test_metrics.seq_report

,Tag,Precision,Recall,F1,support
0,LOC,0.8924,0.9320,0.9117,676
1,MISC,0.7475,0.6214,0.6787,243
2,ORG,0.7817,0.7255,0.7525,459
3,PERS,0.8850,0.8420,0.8630,905
4,micro,0.8547,0.8217,0.8379,2283
5,macro,0.8266,0.7802,0.8015,2283
6,weighted,0.8518,0.8217,0.8356,2283


In [ ]:
ent.entity_prediction['entity'].value_counts()

PERS    1499
LOC      751
ORG      725
O        490
MISC     400
Name: entity, dtype: int64

In [ ]:
Counter(ent.seq_pred)

Counter({'PERS': 861, 'LOC': 706, 'ORG': 426, 'O': 407, 'MISC': 202})

In [ ]:
Counter(ent.seq_true)

Counter({'PERS': 905, 'LOC': 676, 'ORG': 459, 'O': 319, 'MISC': 243})

In [ ]:
ent.entity_prediction.head(50)

,entity,true_token,pred_token,token_id,sen_id,agreement
0,LOC,B-LOC,B-LOC,0,0,True
1,LOC,B-LOC,B-LOC,1,0,True
2,LOC,B-LOC,B-LOC,29,0,True
3,LOC,B-LOC,B-LOC,30,0,True
4,LOC,B-LOC,B-LOC,65,1,True
5,LOC,B-LOC,B-LOC,66,1,True
6,LOC,B-LOC,B-LOC,158,5,True
7,LOC,B-LOC,B-LOC,181,6,True
8,LOC,B-LOC,B-LOC,217,6,True
9,LOC,B-LOC,B-LOC,253,7,True


# Data Annotation

In [ ]:
training_outputs.test_metrics.seq_report

,Tag,Precision,Recall,F1,support
0,LOC,0.8924,0.9320,0.9117,676
1,MISC,0.7475,0.6214,0.6787,243
2,ORG,0.7817,0.7255,0.7525,459
3,PERS,0.8850,0.8420,0.8630,905
4,micro,0.8547,0.8217,0.8379,2283
5,macro,0.8266,0.7802,0.8015,2283
6,weighted,0.8518,0.8217,0.8356,2283


In [ ]:
training_outputs.test_metrics.skl_report

,Tag,Precision,Recall,F1,support
0,B-LOC,0.9021,0.9521,0.9264,668
1,B-MISC,0.8256,0.6851,0.7488,235
2,B-ORG,0.8357,0.7689,0.8009,450
3,B-PERS,0.8865,0.8834,0.8850,858
4,I-LOC,0.8256,0.8554,0.8402,83
5,I-MISC,0.9048,0.4606,0.6104,165
6,I-ORG,0.8150,0.7527,0.7826,275
7,I-PERS,0.9154,0.9111,0.9132,641
8,O,0.9857,0.9923,0.9890,21616
10,macro,0.8774,0.8068,0.8330,24991


In [ ]:
px.scatter(mean_df, x='core_tokens_consistency', y='core_tokens_inconsistency', opacity=0.7)

In [ ]:
df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_analysis_df.jsonl.gz',
    lines=True
    )

In [ ]:
tag_counts

,core_tokens_df,tag,count
0,!,O,1
1,"""",O,397
2,',I-MISC,2
3,',O,23
4,(,O,113
...,...,...,...
9044,يونس,I-LOC,3
9045,يونيس,B-ORG,1
9046,يونيو,O,3
9047,يوهان,B-PERS,1


In [ ]:
tag_counts

,core_tokens_df,tag,count
0,!,O,1
1,"""",O,397
2,',I-MISC,2
3,',O,23
4,(,O,113
...,...,...,...
9044,يونس,I-LOC,3
9045,يونيس,B-ORG,1
9046,يونيو,O,3
9047,يوهان,B-PERS,1


In [ ]:
subwords_df

,core_tokens,tag,count
0,الصالحية,B-LOC,1
1,الصالحية,B-LOC,1
2,الصالحية,B-LOC,1
3,الصالحية,O,1
4,المفرق,B-LOC,1
...,...,...,...
24986,تصميم,O,1
24987,لمبنى,O,1
24988,الوزيرة,O,1
24989,وابن,O,1


# Analysis Data Generation

In [ ]:

config_path = base_folder / 'Final Year Experiments/Class Imbalance/1_fineTuning'
fh = FileHandler(config_path)


In [ ]:
import torch
model_name='arabertv02'
data_name='ANERCorp_CamelLab'
training_outputs = fh.load_pickle(
            f"evalOutputs/{model_name}_{data_name}_regular_outputs.pkl"
        )

load_model_path = fh.file_path / f"trainOutputs/{model_name}_{data_name}_regular.bin"
model = torch.load(load_model_path, map_location=torch.device('cpu'))

In [ ]:
from experiment_utils.model_outputs import ModelOutputWorkflowManager
from experiment_utils.tokenization import TokenizationWorkflowManager
from experiment_utils.evaluation import Metrics

In [ ]:
results_dict = {
            "token_results": training_outputs.test_metrics.skl_results,
            "token_report": training_outputs.test_metrics.skl_report,
            "token_outputs": training_outputs.test_metrics.skl_output,
            "entity_results": training_outputs.test_metrics.seq_results,
            "entity_report": training_outputs.test_metrics.seq_report,
            "entity_outputs": training_outputs.test_metrics.seq_output
        }

results = Metrics.from_dict(results_dict)

In [ ]:
corpora_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
corpora_fh = FileHandler(corpora_path)
corpora  = corpora_fh.load_json('corpora.json')

config_path = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
config_fh = FileHandler(config_path)
config = config_fh.load_yaml('config.yaml')

In [ ]:
from experiment_utils.train import DatasetManager
from experiment_utils.configurations import TrainingConfig, TokenizationConfig
split = 'test'
args = TrainingConfig.from_dict(config.get('training'))
tokenization_config = TokenizationConfig.from_dict(config.get('tokenization'))
data_manager = DatasetManager(corpora, 'ANERCorp_CamelLab', tokenization_config)
batch_sizes = {'train': args.train_batch_size, 'test': args.test_batch_size, 'validation': args.test_batch_size}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-07-28 23:18:50 - INFO - Configuration validated successfully


In [ ]:
NERData = corpora['ANERCorp_CamelLab']
tokenization_outputs = TokenizationWorkflowManager(NERData, tokenization_config)

2024-07-28 23:18:50 - INFO - Loading Tokenizer aubmindlab/bert-base-arabertv02
2024-07-28 23:18:51 - INFO - Loading Preprocessor aubmindlab/bert-base-arabertv02
2024-07-28 23:18:51 - INFO - Processing train split


  0%|          | 0/4149 [00:00<?, ?it/s]

2024-07-28 23:19:04 - INFO - Extracting train subwords


  0%|          | 0/4149 [00:00<?, ?it/s]

2024-07-28 23:19:04 - INFO - Processing test split


  0%|          | 0/961 [00:00<?, ?it/s]

In [ ]:
model_outputs = ModelOutputWorkflowManager(model, device, data_manager, batch_sizes, split)

2024-07-28 23:19:07 - INFO - Specific Split test being processed
2024-07-28 23:19:07 - INFO - Loading Preprocessor: aubmindlab/bert-base-arabertv02
2024-07-28 23:19:07 - INFO - Loading Tokenizer: aubmindlab/bert-base-arabertv02, lower_case: False


  0%|          | 0/121 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from experiment_utils.analysis import DataTransformer, LabelAligner, DataExtractor
from experiment_utils.configurations import UMAPConfig

In [ ]:
from dataclasses import dataclass, asdict

m_outputs = [asdict(sentence) for sentence in model_outputs.test]

In [ ]:
save_model_outputs(model_outputs.test)

In [ ]:
m = load_model_outputs()

In [ ]:
umap_config = UMAPConfig.from_dict(config.get('umap'))


NameError: name 'UMAPConfig' is not defined

In [ ]:
load_states().shape

torch.Size([29711, 768])

In [ ]:
transformer = DataTransformer(umap_config)

In [ ]:
alinger = LabelAligner(results.entity_outputs['y_pred'].copy(), tokenization_outputs.test)

In [ ]:
analysis_data_extractor = DataExtractor(tokenization_outputs.test, model_outputs.test, alinger, transformer)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



UMAP(angular_rp_forest=True, metric='cosine', n_jobs=1, random_state=1, verbose=True)
Wed Jul 24 23:29:02 2024 Construct fuzzy simplicial set
Wed Jul 24 23:29:02 2024 Finding Nearest Neighbors
Wed Jul 24 23:29:02 2024 Building RP forest with 14 trees
Wed Jul 24 23:29:03 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	 6  /  15
	Stopping threshold met -- exiting after 6 iterations
Wed Jul 24 23:29:08 2024 Finished Nearest Neighbor Search
Wed Jul 24 23:29:08 2024 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Wed Jul 24 23:29:32 2024 Finished embedding


In [ ]:
import pandas as pd
df = pd.DataFrame(analysis_data_extractor.analysis_data.to_dict())
df.to_json(config_path / 'test_analysis_df.json', lines=True, orient='records')


In [ ]:
torch.save(analysis_data_extractor.analysis_data.last_hidden_states, config_path / 'last_hidden_states.pt')


PosixPath('/content/drive/My Drive/Final Year Experiments/Thesis-Experiments/scripts/test_analysis_df.json')

In [ ]:
# fig = px.scatter(df, x='x', y='y', color='true_labels', opacity=0.7)


# fig.update_traces(
#             marker=dict(size=2),
#             selected=dict(marker=dict(size=5, opacity=0.9)),  # Selected state
#             unselected=dict(marker=dict(opacity=0.7)),  # Unselected state
#             hoverinfo='all',  # Show all available data on hover
#         )
# fig.update_layout(dragmode='select')  # Enable drag select
# fig.show()


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
tokenization_outputs.test['tokenized_text'][0].labels_df

['[CLS]',
 'B-LOC',
 'B-LOC',
 'O',
 'B-PERS',
 'I-PERS',
 'IGNORED',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'B-LOC',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '[SEP]']

In [ ]:
results.entity_outputs['y_pred'][0]

['B-LOC',
 'B-LOC',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
output[0]

['[CLS]',
 'B-LOC',
 'B-LOC',
 'O',
 'B-PERS',
 'I-PERS',
 'IGNORED',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'B-LOC',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '[SEP]']

In [ ]:
import pandas as pd


df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_analysis_df.jsonl.gz',
    lines=True
    )
# seqeval calculations for entity confusion matrix
confusion_data = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_confusion_data.jsonl.gz',
    lines=True
    )


controid_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_controid_df.jsonl.gz',
    lines=True
    )


entity_prediction = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_entity_prediction.jsonl.gz',
    lines=True
    )


# this is just the x and y coordinates of the pretrained model
pretrained_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_pre_df.jsonl.gz',
    lines=True
    )
# this is exact same information but with extra truth and pred score
token_score_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_token_score_df.jsonl.gz',
    lines=True
    )

# Think about away of loading weights and activations
# think about where the auxilary stuff should be?

In [ ]:
confusion_data['truth'].value_counts()

PERS    905
LOC     676
ORG     459
O       319
MISC    243
Name: truth, dtype: int64

In [ ]:
entity_prediction

,entity,true_token,pred_token,token_id,sen_id,agreement
0,LOC,B-LOC,B-LOC,0,0,True
1,LOC,B-LOC,B-LOC,1,0,True
2,LOC,B-LOC,B-LOC,29,0,True
3,LOC,B-LOC,B-LOC,30,0,True
4,LOC,B-LOC,B-LOC,65,1,True
...,...,...,...,...,...,...
3860,O,B-PERS,B-PERS,25852,958,True
3861,O,I-PERS,I-PERS,25853,958,True
3862,O,B-PERS,I-PERS,25854,958,False
3863,O,I-PERS,I-PERS,25948,960,True


In [ ]:
analysis_df.columns

Index(['global_id', 'token_id', 'word_id', 'sen_id', 'token_ids', 'label_ids',
       'first_tokens_freq', 'first_tokens_consistency',
       'first_tokens_inconsistency', 'words', 'wordpieces', 'tokens',
       'first_tokens', 'truth', 'pred', 'agreement', 'losses', 'x', 'y',
       'tokenization_rate', 'token_entropy', 'word_entropy', 'tr_entity',
       'pr_entity', 'error_type', 'prediction_entropy', 'confidences',
       'variability', 'O', 'B-PERS', 'I-PERS', 'B-ORG', 'I-ORG', 'B-LOC',
       'I-LOC', 'B-MISC', 'I-MISC', '3_clusters', '4_clusters', '9_clusters'],
      dtype='object')

In [ ]:
controid_df

,token_ids,truth,pred,agreement,error_type,centroid,clusters,x,y
0,الصالحية@#0@#1,B-LOC,B-LOC,True,Correct,Centroid-3,cluster-2,7.458792,-4.840983
1,المفرق@#0@#2,B-LOC,B-LOC,True,Correct,Centroid-3,cluster-2,7.550399,-4.857412
2,-@#0@#3,O,O,True,Correct,Centroid-3,cluster-1,1.931367,17.854967
3,غيث@#0@#4,B-PERS,B-PERS,True,Correct,Centroid-3,cluster-0,-4.481947,12.181932
4,الطر@#0@#5,I-PERS,I-PERS,True,Correct,Centroid-3,cluster-0,-6.245377,9.296581
...,...,...,...,...,...,...,...,...,...
74984,C,C,C,C,C,Centroid-9,C,10.065314,-6.042493
74985,C,C,C,C,C,Centroid-9,C,13.743145,-4.104878
74986,C,C,C,C,C,Centroid-9,C,0.448427,3.712907
74987,C,C,C,C,C,Centroid-9,C,1.370305,1.797442


In [ ]:
analysis_df['pred'].iloc[50:100]

50          O
51    IGNORED
52          O
53          O
54          O
55          O
56          O
57          O
58          O
59          O
60          O
61          O
62          O
63          O
64          O
65          O
66          O
67          O
68          O
69          O
70          O
71          O
72      B-LOC
73      B-LOC
74    IGNORED
75          O
76          O
77          O
78          O
79    IGNORED
80    IGNORED
81          O
82          O
83          O
84      [SEP]
85      [CLS]
86          O
87          O
88          O
89          O
90          O
91          O
92          O
93          O
94          O
95          O
96    IGNORED
97          O
98          O
99          O
Name: pred, dtype: object

In [ ]:
len(analysis_df['truth'])

29711

In [ ]:
analysis_df[['tr_entity',
       'pr_entity', 'error_type']]

,tr_entity,pr_entity,error_type
0,[CLS],[CLS],Correct
1,LOC,LOC,Correct
2,LOC,LOC,Correct
3,O,O,Correct
4,PERS,PERS,Correct
...,...,...,...
29706,IGNORED,IGNORED,Correct
29707,PERS,PERS,Correct
29708,PERS,PERS,Correct
29709,O,O,Correct


In [ ]:
confusion_data.head(20)

,truth,pred
0,LOC,LOC
1,LOC,LOC
2,PERS,PERS
3,PERS,PERS
4,LOC,LOC
5,LOC,LOC
6,PERS,O
7,LOC,LOC
8,LOC,LOC
9,O,LOC
